In [1]:
# ============================================================
# CELL 1 (REPLACE) — RAM mode (PHẢI CHẠY TRƯỚC MỌI import HF/transformers/peft)
# Vị trí: TOP notebook
# ============================================================
import os
from pathlib import Path

!df -h /dev/shm

RAM_BASE = Path("/dev/shm/kaggle_ram")
RAM_BASE.mkdir(parents=True, exist_ok=True)

HF_HOME = RAM_BASE / "hf"
HF_HOME.mkdir(parents=True, exist_ok=True)

os.environ["HF_HOME"] = str(HF_HOME)
os.environ["HF_HUB_CACHE"] = str(HF_HOME / "hub")
os.environ["HF_DATASETS_CACHE"] = str(HF_HOME / "datasets")
os.environ["TRANSFORMERS_CACHE"] = str(HF_HOME / "transformers")
os.environ["TORCH_HOME"] = str(RAM_BASE / "torch")
os.environ["XDG_CACHE_HOME"] = str(RAM_BASE / ".cache")

WORKDIR = RAM_BASE / "working"
DATA_DIR = WORKDIR / "data"
TEACH_CACHE_DIR = WORKDIR / "teacher_outputs"
WORKDIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
TEACH_CACHE_DIR.mkdir(parents=True, exist_ok=True)

print("HF_HOME =", os.environ["HF_HOME"])
print("HF_HUB_CACHE =", os.environ["HF_HUB_CACHE"])
print("WORKDIR =", WORKDIR)
print("DATA_DIR =", DATA_DIR)
print("TEACH_CACHE_DIR =", TEACH_CACHE_DIR)


Filesystem      Size  Used Avail Use% Mounted on
shm             114G     0  114G   0% /dev/shm
HF_HOME = /dev/shm/kaggle_ram/hf
HF_HUB_CACHE = /dev/shm/kaggle_ram/hf/hub
WORKDIR = /dev/shm/kaggle_ram/working
DATA_DIR = /dev/shm/kaggle_ram/working/data
TEACH_CACHE_DIR = /dev/shm/kaggle_ram/working/teacher_outputs


In [2]:
# ============================================================
# CELL 2 (REPLACE) — install libs (KHÔNG import transformers ở đây)
# Vị trí: ngay sau CELL 1
# ============================================================
!pip -q install -U --no-cache-dir \
  "transformers>=4.51.0" \
  "accelerate>=0.30.0" \
  "datasets>=2.19.0" \
  "peft>=0.11.0" \
  "trl>=0.9.6" \
  "huggingface_hub>=0.23.0" \
  "tokenizers>=0.21.0" \
  "safetensors>=0.4.3" \
  "sentencepiece" \
  "jsonschema>=4.22.0" \
  "rapidfuzz>=3.9.0" \
  "openpyxl>=3.1.5"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 215.9 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 619.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 432.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 558.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.5/532.5 kB 281.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 260.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.2/507.2 kB 541.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 420.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 244.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ray 2.52.1 requir

In [3]:
# ============================================================
# CELL 3 (REPLACE) — setup PATHS/QWEN32B_PATH
# Vị trí: sau CELL 2
# ============================================================
import os, re, json, random
from glob import glob

DATA_DIR.mkdir(parents=True, exist_ok=True)

def find_qwen32b_path():
    candidates = []
    for p in glob("/kaggle/input/**", recursive=True):
        if os.path.isdir(p):
            low = p.lower()
            if "qwen" in low and ("32b" in low or "32-b" in low):
                if os.path.exists(os.path.join(p, "config.json")):
                    candidates.append(p)
    candidates = sorted(candidates, key=lambda x: len(x))
    return candidates[0] if candidates else None

QWEN32B_PATH = find_qwen32b_path()
print("QWEN32B_PATH =", QWEN32B_PATH)

TEACHERS = {
    "open_finance_8b": "DragonLLM/Llama-Open-Finance-8B",
    "finance_llama3_8b": "instruction-pretrain/finance-Llama3-8B",
    "fingpt_lora_llama3_8b": "FinGPT/fingpt-mt_llama3-8b_lora",
}
FINGPT_BASE = "meta-llama/Meta-Llama-3-8B"

PATHS = {
    "vn_mcocr": "/kaggle/input/vietnamese-receipts-mc-ocr-2021",
    "invoice_ocr": "/kaggle/input/invoice-ocr",
    "hi_quality_invoice": "/kaggle/input/high-quality-invoice-images-for-ocr",
    "gl_xlsx": "/kaggle/input/generalledger/Data file for students.xlsx",
    "transactions_csv": "/kaggle/input/financial-transactions-dataset/financial_transactions.csv",
    "forecast_csv": "/kaggle/input/financial-forecasting-data/simulated_financial_forecasting_data.csv",
    "data_retriever_csv": "/kaggle/input/data-retreiver/Data_ret.csv",
}
print("DATA PATHS OK")


QWEN32B_PATH = /kaggle/input/qwen-3/transformers/32b/1
DATA PATHS OK


In [4]:
# ============================================================
# CELL 4 (REPLACE) — schemas + schema_pass + build_prompt
# Vị trí: sau CELL 3
# ============================================================
from jsonschema import validate
from jsonschema.exceptions import ValidationError
import json

RECEIPT_SCHEMA = {
    "type": "object",
    "properties": {
        "vendor_name": {"type": ["string", "null"]},
        "address": {"type": ["string", "null"]},
        "date": {"type": ["string", "null"]},
        "total_amount": {"type": ["number", "null"]},
        "currency": {"type": ["string", "null"]},
        "confidence": {"type": "number"},
        "flags": {"type": "array", "items": {"type": "string"}},
    },
    "required": ["vendor_name","address","date","total_amount","currency","confidence","flags"],
}

INVOICE_SCHEMA = {
    "type": "object",
    "properties": {
        "vendor_name": {"type": ["string", "null"]},
        "invoice_no": {"type": ["string", "null"]},
        "date": {"type": ["string", "null"]},
        "subtotal": {"type": ["number", "null"]},
        "tax": {"type": ["number", "null"]},
        "total": {"type": ["number", "null"]},
        "currency": {"type": ["string", "null"]},
        "confidence": {"type": "number"},
        "flags": {"type": "array", "items": {"type": "string"}},
    },
    "required": ["vendor_name","invoice_no","date","subtotal","tax","total","currency","confidence","flags"],
}

JOURNAL_SCHEMA = {
    "type": "object",
    "properties": {
        "entries": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "account": {"type": "string"},
                    "debit": {"type": "number"},
                    "credit": {"type": "number"},
                    "memo": {"type": ["string","null"]},
                },
                "required": ["account","debit","credit","memo"],
            },
        },
        "confidence": {"type": "number"},
        "flags": {"type": "array", "items": {"type": "string"}},
    },
    "required": ["entries","confidence","flags"],
}

TASK2SCHEMA = {
    "receipt_extract_text": RECEIPT_SCHEMA,
    "invoice_extract_text": INVOICE_SCHEMA,
    "journal_from_structured_txn": JOURNAL_SCHEMA,
}

def schema_pass(task: str, obj: dict) -> bool:
    try:
        validate(instance=obj, schema=TASK2SCHEMA[task])
        return True
    except ValidationError:
        return False
    except Exception:
        return False

def build_prompt(task: str, input_data):
    if task == "receipt_extract_text":
        return f"""
Extract receipt key fields from Vietnamese text.
Return ONLY valid JSON with fields:
vendor_name,address,date,total_amount,currency,confidence,flags

Receipt Text:
{input_data}
""".strip()

    if task == "invoice_extract_text":
        return f"""
Extract invoice fields from text.
Return ONLY valid JSON with fields:
vendor_name,invoice_no,date,subtotal,tax,total,currency,confidence,flags

Invoice Text:
{input_data}
""".strip()

    if task == "journal_from_structured_txn":
        return f"""
You are an ERP accountant.
Given a structured transaction JSON, propose journal entries.
Return ONLY valid JSON with fields:
- entries: array of objects (account, debit, credit, memo)
- confidence
- flags

Transaction:
{json.dumps(input_data, ensure_ascii=False)}
""".strip()

    raise ValueError("Unknown task")

print("Schemas + prompt builder ready.")


Schemas + prompt builder ready.


In [5]:
# =======================
# CELL 4B (REPLACE)
# Model utils (BF16) + PEFT adapter loader
# Vị trí: thay CELL model utils cũ (trước teacher cell)
# =======================

import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

HF_CACHE_DIR = os.environ.get("HF_HUB_CACHE", None)

def assert_cuda_bf16_ready():
    assert torch.cuda.is_available(), "CUDA is not available. You said H100 runtime."
    name = torch.cuda.get_device_name(0)
    print("CUDA device:", name)
    assert torch.cuda.is_bf16_supported(), "bf16 not supported on this GPU runtime."
    return name

def _pick_attn_impl(prefer_fa2: bool = True) -> str:
    """
    flash_attention_2 requires flash_attn installed.
    If not installed -> fall back to sdpa (always available).
    """
    if prefer_fa2:
        try:
            import flash_attn  # noqa: F401
            return "flash_attention_2"
        except Exception:
            return "sdpa"
    return "sdpa"

def load_bf16_model(repo_or_path: str, prefer_fa2: bool = True):
    """
    Full BF16 weights on GPU. Suitable for H100.
    """
    assert_cuda_bf16_ready()
    attn_impl = _pick_attn_impl(prefer_fa2=prefer_fa2)
    print("Attention implementation:", attn_impl)

    tok = AutoTokenizer.from_pretrained(
        repo_or_path,
        use_fast=True,
        trust_remote_code=True,
        cache_dir=HF_CACHE_DIR,
    )
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    tok.padding_side = "left"

    mdl = AutoModelForCausalLM.from_pretrained(
        repo_or_path,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        attn_implementation=attn_impl,
        cache_dir=HF_CACHE_DIR,
    )
    mdl.eval()
    return tok, mdl

def load_fingpt(adapter_repo: str, base_repo: str, prefer_fa2: bool = True):
    """
    Load base BF16 model + attach LoRA adapter (FinGPT style).
    """
    tok, base = load_bf16_model(base_repo, prefer_fa2=prefer_fa2)
    mdl = PeftModel.from_pretrained(base, adapter_repo)
    mdl.eval()
    return tok, mdl

@torch.no_grad()
def generate_batch(tok, mdl, prompts, max_new_tokens=256):
    enc = tok(prompts, return_tensors="pt", padding=True, truncation=True).to(mdl.device)
    out = mdl.generate(
        **enc,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        top_p=1.0,
        repetition_penalty=1.05,
        pad_token_id=tok.eos_token_id,
    )
    in_len = enc["input_ids"].shape[1]
    gen = out[:, in_len:]
    return tok.batch_decode(gen, skip_special_tokens=True)

print("Model utils ready (BF16 + PEFT adapter loader).")


/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2026-01-22 08:54:44.353036: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769072084.798651     106 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769072084.908022     106 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769072085.885254     106 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769072085.885283     106 computation_pl

Model utils ready (BF16 + PEFT adapter loader).


In [6]:
# ============================================================
# CELL 6 (REPLACE) — loaders (GIỮ NGUYÊN loader của bạn nhưng FIX gold=0 bằng fallback scan columns)
# Vị trí: sau CELL 4B
# ============================================================
import pandas as pd
import os, re, json
from glob import glob

def infer_col(df, candidates):
    cols = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand.lower() in cols:
            return cols[cand.lower()]
    for c in df.columns:
        low = c.lower()
        for cand in candidates:
            if cand.lower() in low:
                return c
    return None

def load_vn_mcocr_cases(limit=2000):
    root = PATHS["vn_mcocr"]
    cases = []

    csv_candidates = [
        os.path.join(root, "mcocr_train_df.csv"),
        os.path.join(root, "mcocr_val_sample_df.csv"),
        os.path.join(root, "results.csv"),
    ]

    for p in csv_candidates:
        if os.path.exists(p):
            df = pd.read_csv(p)

            text_col = infer_col(df, ["text","ocr_text","raw_text","content","transcription"])
            if text_col is None:
                str_cols = [c for c in df.columns if df[c].dtype == "object"]
                if str_cols:
                    text_col = max(str_cols, key=lambda c: df[c].astype(str).str.len().mean())

            seller_col = infer_col(df, ["seller","vendor","vendor_name","merchant","store","shop"])
            addr_col   = infer_col(df, ["address","seller_address","vendor_address"])
            date_col   = infer_col(df, ["date","timestamp","datetime","time"])
            total_col  = infer_col(df, ["total_cost","total","amount","total_amount","sum"])

            def safe_float(x):
                try:
                    if pd.isna(x): 
                        return None
                    s = str(x)
                    s = re.sub(r"[^\d\.\-]", "", s)
                    return float(s) if s else None
                except:
                    return None

            for i, row in df.head(limit).iterrows():
                raw_text = str(row[text_col]) if text_col else ""

                gold = None
                # chỉ tạo gold nếu có ít nhất 1 cột label tồn tại
                if any([seller_col, addr_col, date_col, total_col]):
                    gold = {
                        "vendor_name": str(row[seller_col]) if seller_col and pd.notna(row[seller_col]) else None,
                        "address": str(row[addr_col]) if addr_col and pd.notna(row[addr_col]) else None,
                        "date": str(row[date_col]) if date_col and pd.notna(row[date_col]) else None,
                        "total_amount": safe_float(row[total_col]) if total_col else None,
                        "currency": "VND",
                        "confidence": 0.0,
                        "flags": [],
                    }

                cases.append({
                    "id": f"vn_mcocr_{i}",
                    "task": "receipt_extract_text",
                    "input": raw_text,
                    "gold": gold,
                    "meta": {"source": os.path.basename(p), "cols": list(df.columns)},
                })
            return cases

    return cases

def load_gl_cases(limit=2000):
    xlsx_path = PATHS["gl_xlsx"]
    if not os.path.exists(xlsx_path):
        return []
    xls = pd.ExcelFile(xlsx_path)
    df = pd.read_excel(xlsx_path, sheet_name=xls.sheet_names[0])

    cases = []
    for i, row in df.head(limit).iterrows():
        cases.append({
            "id": f"gl_{i}",
            "task": "journal_from_structured_txn",
            "input": row.to_dict(),
            "gold": None,
            "meta": {"sheet": xls.sheet_names[0]},
        })
    return cases

def load_invoice_ocr_cases(limit=2000):
    root = PATHS["invoice_ocr"]
    if not os.path.exists(root):
        return []

    ann_files = []
    for ext in ["*.json","*.csv"]:
        ann_files += glob(os.path.join(root, "**", ext), recursive=True)

    cases = []
    if ann_files:
        p = ann_files[0]
        if p.endswith(".csv"):
            df = pd.read_csv(p)
            text_col = infer_col(df, ["text","ocr","raw","content"])
            for i, row in df.head(limit).iterrows():
                raw_text = str(row[text_col]) if text_col else ""
                cases.append({
                    "id": f"invoice_ocr_csv_{i}",
                    "task": "invoice_extract_text",
                    "input": raw_text,
                    "gold": None,
                    "meta": {"ann": os.path.basename(p)},
                })
        else:
            with open(p, "r", encoding="utf-8", errors="ignore") as f:
                js = json.load(f)
            items = []
            if isinstance(js, list):
                items = js
            elif isinstance(js, dict):
                for k in ["data","items","annotations","samples"]:
                    if k in js and isinstance(js[k], list):
                        items = js[k]
                        break
            for i, it in enumerate(items[:limit]):
                raw_text = it.get("text") or it.get("ocr_text") or it.get("content") or ""
                cases.append({
                    "id": f"invoice_ocr_json_{i}",
                    "task": "invoice_extract_text",
                    "input": str(raw_text),
                    "gold": None,
                    "meta": {"ann": os.path.basename(p)},
                })
        return cases

    return cases

print("Loaders ready.")


Loaders ready.


In [7]:
# CELL SPLIT (REPLACE) — tách MCOCR (gold) riêng + tạo kd_pool lớn hơn nhưng có kiểm soát
import numpy as np
import pandas as pd
from datetime import datetime, date
import random, json, re
from pathlib import Path

def _json_default(o):
    if isinstance(o, (pd.Timestamp, datetime, date)):
        return o.isoformat()
    if isinstance(o, (np.integer,)):
        return int(o)
    if isinstance(o, (np.floating,)):
        return float(o)
    if isinstance(o, (np.ndarray,)):
        return o.tolist()
    return str(o)

def write_jsonl(path, rows):
    with open(path, "w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False, default=_json_default) + "\n")

# 1) Load dữ liệu
mcocr_cases = load_vn_mcocr_cases(limit=2000)          # tăng để có nhiều gold
invoice_cases = load_invoice_ocr_cases(limit=2000)     # tăng, nhưng thường thiếu gold
gl_cases = load_gl_cases(limit=2000)

# 2) Split MCOCR có gold để EVAL chuẩn
mcocr_gold = [c for c in mcocr_cases if isinstance(c.get("gold"), dict)]
random.shuffle(mcocr_gold)

# Giữ eval_gold đủ lớn để đo; train_gold còn lại để KD/augment nếu muốn
N_EVAL_GOLD = min(500, len(mcocr_gold))
eval_gold_cases = mcocr_gold[:N_EVAL_GOLD]
train_gold_cases = mcocr_gold[N_EVAL_GOLD:]

# 3) Eval set: ưu tiên MCOCR gold + thêm journal để đo schema/journal
eval_cases = []
eval_cases += eval_gold_cases
# thêm journal (không có gold) để đo schema strict
eval_cases += gl_cases[:500]

# (tuỳ chọn) invoice text-only nếu có (nếu input là IMAGE_PATH thì eval text-only LLM sẽ fail => bỏ)
invoice_text_only = [c for c in invoice_cases if isinstance(c.get("input"), str) and not str(c["input"]).startswith("[IMAGE_PATH]")]
eval_cases += invoice_text_only[:300]

random.shuffle(eval_cases)

# 4) KD pool: lấy (train_gold + invoice_text_only + gl) => lớn, đa dạng
kd_pool = []
kd_pool += train_gold_cases
kd_pool += invoice_text_only
kd_pool += gl_cases
random.shuffle(kd_pool)

# giới hạn để iteration ổn; bạn có thể nâng dần (2k, 5k, 10k)
KD_POOL_MAX = min(5000, len(kd_pool))
kd_pool = kd_pool[:KD_POOL_MAX]

print("MCOCR total:", len(mcocr_cases), "| gold:", len(mcocr_gold))
print("EVAL cases:", len(eval_cases), "| EVAL gold:", len(eval_gold_cases))
print("KD pool:", len(kd_pool))

eval_path = str(DATA_DIR / "eval_cases.jsonl")
kd_pool_path = str(DATA_DIR / "kd_pool.jsonl")
write_jsonl(eval_path, eval_cases)
write_jsonl(kd_pool_path, kd_pool)

print("Saved eval_path:", eval_path)
print("Saved kd_pool_path:", kd_pool_path)


MCOCR total: 1155 | gold: 0
EVAL cases: 500 | EVAL gold: 0
KD pool: 2000
Saved eval_path: /dev/shm/kaggle_ram/working/data/eval_cases.jsonl
Saved kd_pool_path: /dev/shm/kaggle_ram/working/data/kd_pool.jsonl


In [8]:
# =======================
# CELL 5 (REPLACE)
# Teacher generation (BF16) - 3 teachers
# Vị trí: ngay sau CELL 4B
# =======================

import json
import torch
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value_1 = user_secrets.get_secret("HF_TOKEN")
HF_TOKEN = os.getenv("HF_TOKEN", secret_value_1)

if HF_TOKEN:
    login(token=HF_TOKEN)
    print("HF login OK")
else:
    print("HF_TOKEN missing -> gated teachers may fail")

TEACH_CACHE_DIR.mkdir(parents=True, exist_ok=True)

def run_teacher(name, repo=None, adapter=None, base=None, batch_size=16, max_new_tokens=256):
    print(f"\n=== Teacher: {name} ===")
    try:
        if adapter and base:
            tok, mdl = load_fingpt(adapter, base)
        else:
            tok, mdl = load_bf16_model(repo)

        with open(kd_pool_path, "r", encoding="utf-8") as f:
            pool = [json.loads(x) for x in f]

        out_path = TEACH_CACHE_DIR / f"{name}.jsonl"
        with open(out_path, "w", encoding="utf-8") as fw:
            for i in range(0, len(pool), batch_size):
                batch = pool[i:i+batch_size]
                prompts = [build_prompt(ex["task"], ex["input"]) for ex in batch]
                texts = generate_batch(tok, mdl, prompts, max_new_tokens=max_new_tokens)

                for ex, t in zip(batch, texts):
                    fw.write(json.dumps({"id": ex["id"], "task": ex["task"], "raw": t}, ensure_ascii=False) + "\n")

        del mdl
        torch.cuda.empty_cache()
        print("Saved:", str(out_path))
        return str(out_path)

    except Exception as e:
        print("FAILED:", name, "| reason:", str(e))
        return None

teacher_paths = {}

teacher_paths["finance_llama3_8b"] = run_teacher(
    "finance_llama3_8b",
    repo=TEACHERS["finance_llama3_8b"],
)

teacher_paths["open_finance_8b"] = run_teacher(
    "open_finance_8b",
    repo=TEACHERS["open_finance_8b"],
)

teacher_paths["fingpt_lora_llama3_8b"] = run_teacher(
    "fingpt_lora_llama3_8b",
    adapter=TEACHERS["fingpt_lora_llama3_8b"],
    base=FINGPT_BASE,
)

print("\nTeacher paths:", teacher_paths)


HF login OK

=== Teacher: finance_llama3_8b ===
CUDA device: NVIDIA H100 80GB HBM3
Attention implementation: sdpa


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/705 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Saved: /dev/shm/kaggle_ram/working/teacher_outputs/fingpt_lora_llama3_8b.jsonl

Teacher paths: {'finance_llama3_8b': '/dev/shm/kaggle_ram/working/teacher_outputs/finance_llama3_8b.jsonl', 'open_finance_8b': '/dev/shm/kaggle_ram/working/teacher_outputs/open_finance_8b.jsonl', 'fingpt_lora_llama3_8b': '/dev/shm/kaggle_ram/working/teacher_outputs/fingpt_lora_llama3_8b.jsonl'}


In [9]:
# ============================================================
# CELL DISTILL (REPLACE) — CLEAN teachers:
#  - parse OK + schema OK
#  - nếu receipt có gold => gần đúng gold (vendor fuzzy + amount close + date exact nếu có)
#  - KHÔNG fill default
# Vị trí: sau teacher cell
# ============================================================
import re, json
from rapidfuzz import fuzz

def _strip_code_fences(s: str) -> str:
    if not s:
        return s
    s = s.strip()
    s = re.sub(r"^```(?:json)?\s*", "", s, flags=re.I)
    s = re.sub(r"\s*```$", "", s)
    return s.strip()

def _try_json_load(s: str):
    try:
        return json.loads(s)
    except Exception:
        return None

def extract_json_robust(text: str):
    if text is None:
        return None
    t = _strip_code_fences(text)

    obj = _try_json_load(t)
    if obj is not None:
        return obj

    m = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", text, flags=re.S | re.I)
    if m:
        obj = _try_json_load(m.group(1))
        if obj is not None:
            return obj

    for m in re.finditer(r"\{.*?\}", t, flags=re.S):
        obj = _try_json_load(m.group(0))
        if obj is not None:
            return obj
    return None

def _to_float(x):
    if x is None:
        return None
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x).replace(",", ".")
    s = re.sub(r"[^0-9\.\-]", "", s)
    try:
        return float(s) if s else None
    except Exception:
        return None

def _to_str(x):
    if x is None:
        return None
    s = str(x).strip()
    return s if s else None

def _to_flags(x):
    if x is None:
        return None
    if isinstance(x, list):
        return [str(v).strip() for v in x if str(v).strip()]
    return [p.strip() for p in re.split(r"[,;\n]+", str(x)) if p.strip()]

def coerce_strict(task: str, obj: dict):
    if not isinstance(obj, dict):
        return None
    if task == "receipt_extract_text":
        return {
            "vendor_name": _to_str(obj.get("vendor_name") or obj.get("vendor") or obj.get("seller")),
            "address": _to_str(obj.get("address") or obj.get("addr")),
            "date": _to_str(obj.get("date")),
            "total_amount": _to_float(obj.get("total_amount") or obj.get("total") or obj.get("amount")),
            "currency": _to_str(obj.get("currency")),
            "confidence": _to_float(obj.get("confidence")),
            "flags": _to_flags(obj.get("flags")),
        }
    if task == "invoice_extract_text":
        return {
            "vendor_name": _to_str(obj.get("vendor_name") or obj.get("vendor") or obj.get("seller")),
            "invoice_no": _to_str(obj.get("invoice_no") or obj.get("invoice_number") or obj.get("invoice")),
            "date": _to_str(obj.get("date")),
            "subtotal": _to_float(obj.get("subtotal")),
            "tax": _to_float(obj.get("tax") or obj.get("vat")),
            "total": _to_float(obj.get("total") or obj.get("total_amount") or obj.get("amount")),
            "currency": _to_str(obj.get("currency")),
            "confidence": _to_float(obj.get("confidence")),
            "flags": _to_flags(obj.get("flags")),
        }
    if task == "journal_from_structured_txn":
        entries = obj.get("entries")
        if not isinstance(entries, list):
            entries = []
        norm = []
        for e in entries:
            if not isinstance(e, dict):
                continue
            norm.append({
                "account": _to_str(e.get("account")) or "",
                "debit": _to_float(e.get("debit")) or 0.0,
                "credit": _to_float(e.get("credit")) or 0.0,
                "memo": _to_str(e.get("memo")),
            })
        return {"entries": norm, "confidence": _to_float(obj.get("confidence")), "flags": _to_flags(obj.get("flags"))}
    return obj

def vendor_sim(a, b):
    if not a or not b:
        return None
    return fuzz.token_set_ratio(str(a), str(b)) / 100.0

def amount_close(g, p, rel_tol=0.02, abs_tol=2000.0):
    if g is None or p is None:
        return None
    try:
        g = float(g); p = float(p)
        return abs(g - p) <= max(abs_tol, rel_tol * max(abs(g), 1.0))
    except Exception:
        return None

def gold_gate_receipt(gold: dict, pred: dict):
    if not isinstance(gold, dict) or pred is None:
        return True
    ok = True
    vs = vendor_sim(gold.get("vendor_name"), pred.get("vendor_name"))
    if vs is not None:
        ok = ok and (vs >= 0.85)
    ac = amount_close(gold.get("total_amount"), pred.get("total_amount"))
    if ac is not None:
        ok = ok and ac
    gd, pd = gold.get("date"), pred.get("date")
    if gd is not None and pd is not None:
        ok = ok and (str(gd).strip() == str(pd).strip())
    return ok

def load_teacher_outputs(path):
    out = {}
    if not path:
        return out
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            r = json.loads(line)
            out[r["id"]] = r
    return out

teacher_outputs = {k: load_teacher_outputs(v) for k, v in teacher_paths.items() if v}

def router_priority(task):
    if task in ["receipt_extract_text", "invoice_extract_text"]:
        return ["open_finance_8b", "finance_llama3_8b", "fingpt_lora_llama3_8b"]
    if task == "journal_from_structured_txn":
        return ["finance_llama3_8b", "open_finance_8b", "fingpt_lora_llama3_8b"]
    return ["finance_llama3_8b", "open_finance_8b", "fingpt_lora_llama3_8b"]

distilled, dropped, picked = [], 0, {}

with open(kd_pool_path, "r", encoding="utf-8") as f:
    pool = [json.loads(x) for x in f]

for ex in pool:
    task, cid, gold = ex["task"], ex["id"], ex.get("gold")
    chosen_obj, chosen_teacher = None, None

    for tname in router_priority(task):
        rec = teacher_outputs.get(tname, {}).get(cid)
        if not rec:
            continue
        raw = rec.get("raw", "")
        obj = extract_json_robust(raw)
        obj = coerce_strict(task, obj) if obj is not None else None
        if obj is None:
            continue
        if not schema_pass(task, obj):
            continue
        if task == "receipt_extract_text" and isinstance(gold, dict):
            if not gold_gate_receipt(gold, obj):
                continue
        chosen_obj, chosen_teacher = obj, tname
        break

    if chosen_obj is None:
        dropped += 1
        continue

    picked[chosen_teacher] = picked.get(chosen_teacher, 0) + 1
    distilled.append({"id": cid, "task": task, "prompt": build_prompt(task, ex["input"]), "answer_json": chosen_obj})

print("KD distilled:", len(distilled), "| dropped:", dropped)
print("picked_by_teacher:", picked)

distill_path = str(DATA_DIR / "distilled_train.jsonl")
with open(distill_path, "w", encoding="utf-8") as f:
    for r in distilled:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print("Saved:", distill_path)


KD distilled: 1198 | dropped: 802
picked_by_teacher: {'finance_llama3_8b': 1180, 'open_finance_8b': 18}
Saved: /dev/shm/kaggle_ram/working/data/distilled_train.jsonl


In [10]:
# ============================================================
# CELL TRAIN (REPLACE) — BF16 Qwen3-32B + LoRA SFT (optim chuẩn)
# Vị trí: sau distill
# ============================================================
from datasets import Dataset
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from transformers import TrainingArguments
from pathlib import Path
import json

assert QWEN32B_PATH is not None, "Cannot find Qwen3-32B in /kaggle/input"

qwen_tok, qwen_base = load_bf16_model(QWEN32B_PATH)

def guess_lora_targets(model):
    names = set()
    for n, _ in model.named_modules():
        if any(k in n for k in ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]):
            names.add(n.split(".")[-1])
    return sorted(list(names)) if names else ["q_proj","k_proj","v_proj","o_proj"]

targets = guess_lora_targets(qwen_base)
print("LoRA targets:", targets)

lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=targets,
)

student = get_peft_model(qwen_base, lora_cfg)
student.print_trainable_parameters()

rows = []
with open(distill_path, "r", encoding="utf-8") as f:
    for line in f:
        r = json.loads(line)
        rows.append({"text": r["prompt"] + "\n\n" + json.dumps(r["answer_json"], ensure_ascii=False)})

if len(rows) == 0:
    raise RuntimeError("distilled_train.jsonl is empty (distill produced 0)")

train_ds = Dataset.from_list(rows)

args = TrainingArguments(
    output_dir=str(WORKDIR / "student_ckpt"),
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    num_train_epochs=1,      # tăng lên 2-3 CHỈ KHI EVAL-GOLD tăng
    warmup_steps=50,
    logging_steps=10,
    save_steps=200,
    bf16=True,
    optim="adamw_torch",
    report_to="none",
)

trainer = SFTTrainer(
    model=student,
    args=args,
    train_dataset=train_ds,
    processing_class=qwen_tok,
)

trainer.train()

ADAPTER_DIR = Path(WORKDIR) / "outputs" / "adapters" / "student_adapter"
ADAPTER_DIR.mkdir(parents=True, exist_ok=True)
trainer.model.save_pretrained(str(ADAPTER_DIR))
qwen_tok.save_pretrained(str(ADAPTER_DIR))

assert (ADAPTER_DIR / "adapter_config.json").exists(), f"Missing adapter_config.json in {ADAPTER_DIR}"
print("✅ Saved student adapter:", str(ADAPTER_DIR))


/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

CUDA device: NVIDIA H100 80GB HBM3
Attention implementation: sdpa


Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

LoRA targets: ['down_proj', 'gate_proj', 'k_proj', 'o_proj', 'q_proj', 'up_proj', 'v_proj']
trainable params: 134,217,728 || all params: 32,896,340,992 || trainable%: 0.4080


/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:2111: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(


...

Tokenizing train dataset:   0%|          | 0/1198 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1198 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,1.525800
20,1.354000
30,0.651800
40,0.208100
50,0.156300
60,0.138200
70,0.128100
80,0.124100
90,0.116500
100,0.119600


✅ Saved student adapter: /dev/shm/kaggle_ram/working/outputs/adapters/student_adapter


In [11]:
# ============================================================
# CELL EVAL (B-STRICT) (REPLACE) — STRICT, score gold receipts, NO default fill
# Vị trí: sau TRAIN
# ============================================================
import json, re
from collections import defaultdict
from rapidfuzz import fuzz
from peft import PeftModel

def _strip_code_fences(s: str) -> str:
    if not s:
        return s
    s = s.strip()
    s = re.sub(r"^```(?:json)?\s*", "", s, flags=re.I)
    s = re.sub(r"\s*```$", "", s)
    return s.strip()

def _try_json_load(s: str):
    try:
        return json.loads(s)
    except Exception:
        return None

def extract_json_robust(text: str):
    if text is None:
        return None
    t = _strip_code_fences(text)
    obj = _try_json_load(t)
    if obj is not None:
        return obj
    m = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", text, flags=re.S | re.I)
    if m:
        obj = _try_json_load(m.group(1))
        if obj is not None:
            return obj
    for m in re.finditer(r"\{.*?\}", t, flags=re.S):
        obj = _try_json_load(m.group(0))
        if obj is not None:
            return obj
    return None

def _to_float(x):
    if x is None:
        return None
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x).replace(",", ".")
    s = re.sub(r"[^0-9\.\-]", "", s)
    try:
        return float(s) if s else None
    except Exception:
        return None

def _to_str(x):
    if x is None:
        return None
    s = str(x).strip()
    return s if s else None

def _to_flags(x):
    if x is None:
        return None
    if isinstance(x, list):
        return [str(v).strip() for v in x if str(v).strip()]
    return [p.strip() for p in re.split(r"[,;\n]+", str(x)) if p.strip()]

def coerce_strict(task: str, obj: dict):
    if not isinstance(obj, dict):
        return None
    if task == "receipt_extract_text":
        return {
            "vendor_name": _to_str(obj.get("vendor_name") or obj.get("vendor") or obj.get("seller")),
            "address": _to_str(obj.get("address") or obj.get("addr")),
            "date": _to_str(obj.get("date")),
            "total_amount": _to_float(obj.get("total_amount") or obj.get("total") or obj.get("amount")),
            "currency": _to_str(obj.get("currency")),
            "confidence": _to_float(obj.get("confidence")),
            "flags": _to_flags(obj.get("flags")),
        }
    if task == "invoice_extract_text":
        return {
            "vendor_name": _to_str(obj.get("vendor_name") or obj.get("vendor") or obj.get("seller")),
            "invoice_no": _to_str(obj.get("invoice_no") or obj.get("invoice_number") or obj.get("invoice")),
            "date": _to_str(obj.get("date")),
            "subtotal": _to_float(obj.get("subtotal")),
            "tax": _to_float(obj.get("tax") or obj.get("vat")),
            "total": _to_float(obj.get("total") or obj.get("total_amount") or obj.get("amount")),
            "currency": _to_str(obj.get("currency")),
            "confidence": _to_float(obj.get("confidence")),
            "flags": _to_flags(obj.get("flags")),
        }
    if task == "journal_from_structured_txn":
        entries = obj.get("entries")
        if not isinstance(entries, list):
            entries = []
        norm = []
        for e in entries:
            if not isinstance(e, dict):
                continue
            norm.append({
                "account": _to_str(e.get("account")) or "",
                "debit": _to_float(e.get("debit")) or 0.0,
                "credit": _to_float(e.get("credit")) or 0.0,
                "memo": _to_str(e.get("memo")),
            })
        return {"entries": norm, "confidence": _to_float(obj.get("confidence")), "flags": _to_flags(obj.get("flags"))}
    return obj

def vendor_sim(a, b):
    if not a or not b:
        return None
    return fuzz.token_set_ratio(str(a), str(b)) / 100.0

def amount_close(g, p, rel_tol=0.02, abs_tol=2000.0):
    if g is None or p is None:
        return None
    try:
        g = float(g); p = float(p)
        return abs(g - p) <= max(abs_tol, rel_tol * max(abs(g), 1.0))
    except Exception:
        return None

def eval_b_strict(tok, mdl, cases, batch_size=32, max_new_tokens=256):
    by_task = defaultdict(list)
    for c in cases:
        by_task[c["task"]].append(c)

    per_task = {}
    for task, exs in by_task.items():
        n = len(exs)
        json_valid = 0
        schema_ok = 0
        for i in range(0, n, batch_size):
            batch = exs[i:i+batch_size]
            prompts = [build_prompt(task, ex["input"]) for ex in batch]
            texts = generate_batch(tok, mdl, prompts, max_new_tokens=max_new_tokens)
            for ex, t in zip(batch, texts):
                raw = extract_json_robust(t)
                obj = coerce_strict(task, raw) if raw is not None else None
                if obj is not None:
                    json_valid += 1
                    if schema_pass(task, obj):
                        schema_ok += 1
        per_task[task] = {"n": n, "json_valid_rate": json_valid/max(1,n), "schema_pass_rate": schema_ok/max(1,n)}

    gold_receipts = [
        c for c in cases
        if c["task"] == "receipt_extract_text"
        and isinstance(c.get("gold"), dict)
        and any(c["gold"].get(k) not in [None, "", "nan", "NaN"] for k in ["vendor_name","date","total_amount","address"])
    ]
    receipt_gold = None
    if gold_receipts:
        vs_list, tc_list, de_list = [], [], []
        n = len(gold_receipts)
        for i in range(0, n, batch_size):
            batch = gold_receipts[i:i+batch_size]
            prompts = [build_prompt("receipt_extract_text", ex["input"]) for ex in batch]
            texts = generate_batch(tok, mdl, prompts, max_new_tokens=max_new_tokens)
            for ex, t in zip(batch, texts):
                gold = ex["gold"]
                raw = extract_json_robust(t)
                obj = coerce_strict("receipt_extract_text", raw) if raw is not None else None
                vs = vendor_sim(gold.get("vendor_name"), obj.get("vendor_name") if obj else None)
                if vs is not None:
                    vs_list.append(vs)
                ac = amount_close(gold.get("total_amount"), obj.get("total_amount") if obj else None)
                if ac is not None:
                    tc_list.append(1.0 if ac else 0.0)
                gd, pd = gold.get("date"), (obj.get("date") if obj else None)
                if gd is not None and pd is not None:
                    de_list.append(1.0 if str(gd).strip() == str(pd).strip() else 0.0)

        def avg(xs): return float(sum(xs)/len(xs)) if xs else None
        receipt_gold = {"n": n, "vendor_sim_avg": avg(vs_list), "total_close_rate": avg(tc_list), "date_exact_rate": avg(de_list)}

    return {"per_task": per_task, "receipt_gold": receipt_gold}

with open(eval_path, "r", encoding="utf-8") as f:
    eval_cases = [json.loads(x) for x in f]

# BASE
base_tok, base_mdl = load_bf16_model(QWEN32B_PATH)
base_rep = eval_b_strict(base_tok, base_mdl, eval_cases, batch_size=32, max_new_tokens=256)
print("BASE per_task schema:", {k: v["schema_pass_rate"] for k, v in base_rep["per_task"].items()})
print("BASE receipt_gold:", base_rep["receipt_gold"])

# STUDENT
student_tok, student_base = load_bf16_model(QWEN32B_PATH)
student_mdl = PeftModel.from_pretrained(student_base, str(ADAPTER_DIR))
student_mdl.eval()

student_rep = eval_b_strict(student_tok, student_mdl, eval_cases, batch_size=32, max_new_tokens=256)
print("STUDENT per_task schema:", {k: v["schema_pass_rate"] for k, v in student_rep["per_task"].items()})
print("STUDENT receipt_gold:", student_rep["receipt_gold"])

report = {
    "base": base_rep,
    "student": student_rep,
    "meta": {
        "eval_path": str(eval_path),
        "strict": True,
        "no_default_fill": True,
        "batch_size": 8,
        "max_new_tokens": 256,
        "note": "If receipt_gold is None => MCOCR has no usable gold columns; must fix dataset/loader first.",
    },
}

report_path = str(DATA_DIR / "eval_report_b_strict.json")
with open(report_path, "w", encoding="utf-8") as f:
    json.dump(report, f, ensure_ascii=False, indent=2)

print("Saved report:", report_path)


CUDA device: NVIDIA H100 80GB HBM3
Attention implementation: sdpa


Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


OutOfMemoryError: CUDA out of memory. Tried to allocate 266.00 MiB. GPU 0 has a total capacity of 79.44 GiB of which 260.62 MiB is free. Process 4495 has 79.18 GiB memory in use. Of the allocated memory 77.75 GiB is allocated by PyTorch, and 716.43 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# ============================================================
# CELL SAVE (ADD NEW, cuối notebook) — export adapter + tokenizer + lineage + zip
# Vị trí: cuối notebook
# ============================================================
import json, shutil
from pathlib import Path

REL = Path("/kaggle/working") / "release" / "qwen3-32b-accounting-distilled-v0.1.0"
REL.mkdir(parents=True, exist_ok=True)

ADAPTER_OUT = REL / "adapters"
TOKEN_OUT  = REL / "tokenizer"
ADAPTER_OUT.mkdir(parents=True, exist_ok=True)
TOKEN_OUT.mkdir(parents=True, exist_ok=True)

trainer.model.save_pretrained(str(ADAPTER_OUT))
qwen_tok.save_pretrained(str(TOKEN_OUT))

lineage = {
    "model_name": "qwen3-32b-accounting-distilled",
    "version": "v0.1.0",
    "method": "KD(clean filter) + LoRA SFT(BF16) tuned by GOLD-metric",
    "teachers": TEACHERS,
    "paths": {
        "distill_path": str(distill_path),
        "adapter_dir": str(ADAPTER_OUT),
        "tokenizer_dir": str(TOKEN_OUT),
        "eval_report": str(DATA_DIR / "eval_report_b_strict.json"),
    },
}
with open(REL / "lineage.json", "w", encoding="utf-8") as f:
    json.dump(lineage, f, ensure_ascii=False, indent=2)

zip_path = shutil.make_archive(str(REL), "zip", root_dir=str(REL))
print("✅ RELEASE folder:", REL)
print("✅ ZIP:", zip_path)


In [ ]:
# ============================================================
# CELL SAVE (ADD NEW, cuối notebook) — export adapter + tokenizer + lineage + zip
# Vị trí: cuối notebook
# ============================================================
import json, shutil
from pathlib import Path

REL = Path("/kaggle/working") / "release" / "qwen3-32b-accounting-distilled-v0.1.0"
REL.mkdir(parents=True, exist_ok=True)

ADAPTER_OUT = REL / "adapters"
TOKEN_OUT  = REL / "tokenizer"
ADAPTER_OUT.mkdir(parents=True, exist_ok=True)
TOKEN_OUT.mkdir(parents=True, exist_ok=True)

trainer.model.save_pretrained(str(ADAPTER_OUT))
qwen_tok.save_pretrained(str(TOKEN_OUT))

lineage = {
    "model_name": "qwen3-32b-accounting-distilled",
    "version": "v0.1.0",
    "method": "KD(clean filter) + LoRA SFT(BF16) tuned by GOLD-metric",
    "teachers": TEACHERS,
    "paths": {
        "distill_path": str(distill_path),
        "adapter_dir": str(ADAPTER_OUT),
        "tokenizer_dir": str(TOKEN_OUT),
        "eval_report": str(DATA_DIR / "eval_report_b_strict.json"),
    },
}
with open(REL / "lineage.json", "w", encoding="utf-8") as f:
    json.dump(lineage, f, ensure_ascii=False, indent=2)

zip_path = shutil.make_archive(str(REL), "zip", root_dir=str(REL))
print("✅ RELEASE folder:", REL)
print("✅ ZIP:", zip_path)
